In [1]:
import pandas as pd
import os

In [2]:
data_dir = '../data'

fire_df = pd.read_csv(os.path.join(data_dir, 'fire2.csv'))
fire_df.head()

,lat_bounds,lon_bounds,year,month,fraction_of_burnable_area,burned_area,CL
0,52.0,-10.00,2010,6,0.805087,0.0,1
1,52.0,-9.75,2010,6,0.937479,0.0,0
2,52.0,-9.50,2010,6,0.998650,0.0,1
3,52.0,-9.25,2010,6,0.996290,0.0,1
4,52.0,-9.00,2010,6,0.979073,0.0,1


In [3]:
fire_df['month'].unique()

array([6, 4, 5, 7])

In [4]:
fire_df['month'] = fire_df['month'] + 1
fire_df.head()

,lat_bounds,lon_bounds,year,month,fraction_of_burnable_area,burned_area,CL
0,52.0,-10.00,2010,7,0.805087,0.0,1
1,52.0,-9.75,2010,7,0.937479,0.0,0
2,52.0,-9.50,2010,7,0.998650,0.0,1
3,52.0,-9.25,2010,7,0.996290,0.0,1
4,52.0,-9.00,2010,7,0.979073,0.0,1


In [5]:
fire_df['month'].unique()

array([7, 5, 6, 8])

In [6]:
landcover_df = pd.read_csv(os.path.join(data_dir, 'satellite_landcover_dataset.csv'))
landcover_df

,lat,lon,lat_bounds,lon_bounds,processed_flag,lccs_class,current_pixel_state,observation_count,change_count,time
0,52.00,-10.00,52.125000,-10.125000,1.0,130,1.0,119,0,2001
1,39.25,29.50,39.375000,29.375000,1.0,100,1.0,333,0,2001
2,39.25,29.75,39.375000,29.625000,1.0,100,1.0,302,1,2001
3,39.25,30.00,39.375000,29.875000,1.0,70,1.0,276,0,2001
4,39.25,30.25,39.375000,30.125000,1.0,10,1.0,332,0,2001
...,...,...,...,...,...,...,...,...,...,...
742275,39.25,29.50,39.375000,29.375000,1.0,100,1.0,333,0,2020
742276,39.25,29.25,39.375000,29.125000,1.0,70,1.0,250,0,2020
742277,39.25,29.00,39.375000,28.875000,1.0,70,1.0,271,0,2020
742278,39.25,42.00,39.375000,41.875000,1.0,10,1.0,325,0,2020


In [7]:
filtered_landcover_df = landcover_df[['lat',
                                     'lon',
                                     'lccs_class',
                                     'time']]

filtered_landcover_df = filtered_landcover_df.rename(columns={'time': 'year'})

filtered_landcover_df

,lat,lon,lccs_class,year
0,52.00,-10.00,130,2001
1,39.25,29.50,100,2001
2,39.25,29.75,100,2001
3,39.25,30.00,70,2001
4,39.25,30.25,10,2001
...,...,...,...,...
742275,39.25,29.50,100,2020
742276,39.25,29.25,70,2020
742277,39.25,29.00,70,2020
742278,39.25,42.00,10,2020


In [8]:
climate_df = pd.read_csv(os.path.join(data_dir, '../data/climate.csv'))
climate_df.head()

,Unnamed: 0,lon,lat,year,tas_5,tasmax_5,pr_5,tas_6,tasmax_6,pr_6,tas_7,tasmax_7,pr_7,tas_8,tasmax_8,pr_8
0,0,-10.0,33.00,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-10.0,33.25,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-10.0,33.50,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,-10.0,33.75,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,-10.0,34.00,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
climate_df.shape

(352583, 16)

the folowing code is for restructuring climate data for easier joining on other two datasets

In [10]:
climate_df = climate_df.dropna().drop('Unnamed: 0', axis =1)

import pandas as pd

# assuming your data is stored in a variable named df
climate_df = pd.melt(climate_df, id_vars=['lon', 'lat', 'year'], var_name='variable', value_name='value')
climate_df[['variable', 'month']] = climate_df['variable'].str.extract('(\D+)(\d+)', expand=True)
climate_df['month'] = climate_df['month'].astype(int)
climate_df = climate_df.pivot_table(index=['lon', 'lat', 'year', 'month'], columns='variable', values='value').reset_index()
climate_df.columns.name = None
climate_df.rename(columns=lambda x: x.rstrip('_'), inplace=True)

climate_df

,lon,lat,year,month,pr,tas,tasmax
0,-10.0,51.75,2001,5,39.000000,12.300000,16.500000
1,-10.0,51.75,2001,6,59.900000,13.400001,17.000000
2,-10.0,51.75,2001,7,79.000000,15.300000,18.800001
3,-10.0,51.75,2001,8,102.100000,15.300000,18.900000
4,-10.0,51.75,2002,5,153.700000,11.300000,14.800000
...,...,...,...,...,...,...,...
1056395,50.0,52.00,2018,8,8.300000,21.500000,28.500000
1056396,50.0,52.00,2019,5,29.500000,18.800001,25.200000
1056397,50.0,52.00,2019,6,22.800001,22.400000,29.300001
1056398,50.0,52.00,2019,7,40.500000,22.400000,28.600000


In [11]:
fire_landcover = fire_df.merge(
    filtered_landcover_df,
    left_on=['lat_bounds', 'lon_bounds', 'year'],
    right_on=['lat', 'lon', 'year']
    )

fire_landcover = fire_landcover.drop(['lat', 'lon'], axis =1)
fire_landcover

,lat_bounds,lon_bounds,year,month,fraction_of_burnable_area,burned_area,CL,lccs_class
0,52.0,-10.0,2010,7,0.805087,0.0,1,130
1,52.0,-10.0,2010,7,0.805087,0.0,1,130
2,52.0,-10.0,2010,5,0.805087,0.0,1,130
3,52.0,-10.0,2010,5,0.805087,0.0,1,130
4,52.0,-10.0,2010,8,0.805087,0.0,1,130
...,...,...,...,...,...,...,...,...
3959275,33.0,50.0,2015,5,0.973475,0.0,1,30
3959276,33.0,50.0,2015,7,0.973475,0.0,1,30
3959277,33.0,50.0,2015,7,0.973475,0.0,1,30
3959278,33.0,50.0,2015,6,0.973475,0.0,1,30


In [12]:
merged_df = fire_landcover.merge(
    climate_df,
    left_on=['lat_bounds', 'lon_bounds', 'year', 'month'],
    right_on=['lat', 'lon', 'year', 'month']
    )

merged_df = merged_df.drop(['lat', 'lon'], axis =1).reset_index(drop=True).drop_duplicates()
merged_df

,lat_bounds,lon_bounds,year,month,fraction_of_burnable_area,burned_area,CL,lccs_class,pr,tas,tasmax
0,52.0,-10.00,2010,7,0.805087,0.0,1,130,131.5,15.700000,18.800001
2,52.0,-10.00,2010,5,0.805087,0.0,1,130,58.9,11.000000,15.000000
4,52.0,-10.00,2010,8,0.805087,0.0,1,130,35.8,14.700000,18.400000
6,52.0,-10.00,2010,6,0.805087,0.0,1,130,49.0,14.900001,19.100000
8,52.0,-9.75,2010,7,0.937479,0.0,0,210,168.1,14.600000,17.400000
...,...,...,...,...,...,...,...,...,...,...,...
3875214,33.0,49.75,2015,6,0.983409,0.0,1,11,25.5,23.800001,33.100002
3875216,33.0,50.00,2015,8,0.973475,0.0,1,30,1.7,24.800001,34.000000
3875218,33.0,50.00,2015,5,0.973475,0.0,1,30,64.9,17.500000,25.500000
3875220,33.0,50.00,2015,7,0.973475,0.0,1,30,29.7,25.600000,34.500000


In [13]:
merged_df.isna().sum()

lat_bounds                   0
lon_bounds                   0
year                         0
month                        0
fraction_of_burnable_area    0
burned_area                  0
CL                           0
lccs_class                   0
pr                           0
tas                          0
tasmax                       0
dtype: int64

In [14]:
merged_df.to_csv(os.path.join(data_dir, 'merged_df.csv'), index=False)